#### Using a set of functions to help tidy/clean up the scraped recipe html soup info. and retrieve relevant parameters of interest

In [1]:
from __future__ import print_function, division

# import matplotlib
# % matplotlib inline

import pandas as pd
import numpy as np
# import datetime as dt
# import matplotlib.pyplot as plt


from bs4 import BeautifulSoup
import requests
import re
import time
import sys

from recipe_scrapers import scrap_me
import pickle

In [ ]:
# scrapeInt = [int(i) for i in np.round(np.linspace(0, 10081,100))]

In [2]:
# function to find index of capitalized character in string of words

def getCAPcharIndices(s):
    return [i for i, c in enumerate(s) if c.isupper()]

In [3]:
# 27Nov2016 edited

def getBracketIndices(s):
    return [i for i, c in enumerate(s) if c=='(']

In [4]:
# this will run through all recipes in list of recipes and strip-clean the ingredients...
## older version -- with some imperfections but still pretty good

def CleanupRecipeIngredients0(Recipes):
    cnt=0
    for rcp in Recipes:
        rcp_ingd = rcp.ingredients()
        tmp=[]
    #     print(rcp_ingd)
        for ingd in rcp_ingd:
    #         print(ingd)
            idx = getCAPcharIndices(ingd)
    #         print(idx)
            if idx==[]:
    #             print(ingd)
                tmp.append(ingd)
            else:
    #             print(ingd[:idx[0]])
                #tmp.append(ingd[:idx[0]])
                tmp.append(ingd[:idx[0]-1]) # 11Feb2017
    #         print(tmp)
        Recipes[cnt].ingredients1=tmp
        cnt +=1
    return Recipes

In [5]:
## 24-27Nov2016 edited -- tried to add additional parameters -- some imperfections no less
def CleanupRecipeIngredients(Recipes):
    cnt=0
    for rcp in Recipes:
        #print(cnt)
        rcp_ingd = rcp.ingredients()
        tmp=[]
        #print(rcp_ingd)
        for ingd in rcp_ingd:
            #print(ingd)
            idx = getCAPcharIndices(ingd)
            idbk = getBracketIndices(ingd)
            #print(idbk)
            #print(idx)
            if idx==[] or len(idx)==1:
                #print(ingd)
                tmp.append(ingd.strip())
            else:               
                #print(ingd[:idx[0]])
                #tmp.append(ingd[:idx[0]])
                ix = list(sum(np.where(np.array(idx)>15)))
        
                #if len(idx)==1:
                #    tmp.append(ingd[:idx[0]].strip())
                
                if len(idx)>1 and idbk!=[]:
                    if idx[0]<idbk[0]:
                        tmp.append(ingd.strip())

                elif len(idx)>1 and (idx[0]<3 or idx[0]<14):
                    tmp.append(ingd[:idx[1]].strip())
                    
                elif len(idx)>1 and idx[0]<=14:
                    tmp.append(ingd[:idx[0]].strip())
                
                elif ix!=[] :
                    #print(ingd)
                    tmp.append(ingd[:idx[ix[0]]].strip())
                    #print(tmp)
                    
        Recipes[cnt].ingredients2=tmp
        cnt +=1
    return Recipes

In [ ]:
# CleanupRecipeIngredients(Recipes)

In [6]:
def CleanupRecipeInstructions(Recipes):
    cnt=0
    for rcp in Recipes:
        # print (rcp.instructions())
        updateInstruct = rcp.instructions().replace('\n',' ')
        
        Recipes[cnt].instructions2 = updateInstruct
        cnt +=1
    return Recipes

In [ ]:
# CleanupRecipeInstructions(Recipes);

In [ ]:
# for sI in range(40,100):#range(1,len(scrapeInt)): #80,len(scrapeInt)
#     filename = ("scrapedInfo_BBCgdfd_" + format(scrapeInt[sI-1] ) + "to" + format(scrapeInt[sI] ) + "recipes.p")
#     #print(filename)
#     print(format(sI) +': CLEANING Ingredients & Instructions') #== pickle-load : ' +filename)
    
#     with open(filename, "rb") as f:
#         Recipes = pickle.load(f)
    
#     CleanupRecipeIngredients(Recipes)
#     CleanupRecipeInstructions(Recipes);
    
#     with open(filename, "wb") as f:
#         pickle.dump(Recipes, f)
#     print('Cleanedup -- pickle-dumped : ' +filename)

In [7]:
## check prep-cook-times HTML TAGs
def checkPrepCookFullClassTags(recipeInfo):
    prepNone, cookNone, fullNone = (recipeInfo.soup.find('span', {'class': 'recipe-details__cooking-time-prep'}) == None, #false=value
                                    recipeInfo.soup.find('span', {'class': 'recipe-details__cooking-time-cook'}) == None,
                                    recipeInfo.soup.find('span', {'class': 'recipe-details__cooking-time-full'}) == None) #true=empty
    return prepNone, cookNone, fullNone

In [8]:
def checkNgetPrepTime(recipeInfo,prepNone):
    if prepNone == False:
        #print('has info')
        prep_hr = recipeInfo.soup.find('span', {'class': 'recipe-details__cooking-time-prep'}).find_all('span',{'class':'hrs'})  
        if prep_hr==[]:
            prep_hr_val = np.nan
        else:
#             prep_hr_val = int(prep_hr[0].text.replace(' hrs','').replace(' hr',''))
            if len(prep_hr)==1:
                prep_hr_val = int(prep_hr[0].text.replace(' hrs','').replace(' hr',''))
            else:
                prep_hr_val = int(prep_hr[len(prep_hr)-1].text.replace(' hrs','').replace(' hr',''))


        prep_min = recipeInfo.soup.find('span', {'class': 'recipe-details__cooking-time-prep'}).find_all('span',{'class':'mins'})
        if prep_min==[]:
            prep_min_val = np.nan

        else:
#             prep_min_val = int(prep_min[0].text.replace(' mins','').replace(' min',''))
            if len(prep_min)==1:
                prep_min_val = int(prep_min[0].text.replace(' mins','').replace(' min',''))
            else:
                prep_min_val = int(prep_min[len(prep_min)-1].text.replace(' mins','').replace(' min',''))

        preptime = np.nansum(np.array([prep_min_val, prep_hr_val*60]) )
        return preptime

In [9]:
def checkNgetCookingTime(recipeInfo,cookNone):

    if cookNone == False:

        cook_hr = recipeInfo.soup.find('span', {'class': 'recipe-details__cooking-time-cook'}).find_all('span',{'class':'hrs'})
        if cook_hr==[]:
            cook_hr_val = np.nan
        else:
#             cook_hr_val = int(cook_hr[0].text.replace(' hrs','').replace(' hr',''))
            if len(cook_hr)==1:
                cook_hr_val = int(cook_hr[0].text.replace(' hrs','').replace(' hr',''))
            else:
                cook_hr_val = int(cook_hr[len(cook_hr)-1].text.replace(' hrs','').replace(' hr',''))

        cook_min = recipeInfo.soup.find('span', {'class': 'recipe-details__cooking-time-cook'}).find_all('span',{'class':'mins'})
        if cook_min==[]:
            cook_min_val = np.nan
            
        else:
#             cook_min_val = int(cook_min[0].text.replace(' mins','').replace(' min',''))
            if len(cook_min)==1:
                cook_min_val = int(cook_min[0].text.replace(' mins','').replace(' min',''))
            else:
                cook_min_val = int(cook_min[len(cook_min)-1].text.replace(' mins','').replace(' min',''))

        cooktime = np.nansum(np.array([cook_min_val, cook_hr_val*60]) ) 
        return cooktime

In [10]:
def checkNgetFullCookingPrepInfo(recipeInfo, fullNone):
    if fullNone == False:
        prepInfo = recipeInfo.soup.find('span', {'class': 'recipe-details__cooking-time-full'}).text
        #print(prepInfo)
        return prepInfo
    else:
        return None

In [11]:
def checkNgetPrepCookFullTimesInfo(Recipes):
    cnt=0
    for recipeInfo in Recipes:
#         print(cnt)
        
        prepNone, cookNone, fullNone = checkPrepCookFullClassTags(recipeInfo)

        Time2prep_mins = checkNgetPrepTime(recipeInfo,prepNone)
        Time2cook_mins = checkNgetCookingTime(recipeInfo,cookNone)
        prepInfo = checkNgetFullCookingPrepInfo(recipeInfo,fullNone)

        if Time2prep_mins != np.nan:
            Recipes[cnt].prepTime_mins = Time2prep_mins
        else:
            Recipes[cnt].prepTime_mins = np.nan

        if Time2cook_mins != np.nan:    
            Recipes[cnt].cookTime_mins = Time2cook_mins
        else:
            Recipes[cnt].cookTime_mins = np.nan

        if recipeInfo != None:
            Recipes[cnt].prepInfo = prepInfo
        else:
            Recipes[cnt].prepInfo = None
        
#         if hasattr(recipeInfo, 'recipeInfo') == True:
#             del recipeInfo.recipeInfo
                
        cnt +=1

    return Recipes

In [12]:
def GetRecipeDifficulty(Recipes):
    cnt=0
    for recipeInfo in Recipes:
        prepDifficulty = recipeInfo.soup.find('span', {'class': 'recipe-details__text'}).text.strip(' ')
        
        Recipes[cnt].prepDifficulty = prepDifficulty
        cnt +=1
        
    return Recipes

In [13]:
def GetRecipeYield(Recipes):
    cnt=0
    for recipeInfo in Recipes:
        recpYield = recipeInfo.soup.find('span', {'itemprop': 'recipeYield'}).text.strip(' ')
        
        Recipes[cnt].recpYield = recpYield
        cnt +=1
        
    return Recipes

In [14]:
def GetExtraRecipeInfo(Recipes):
    cnt=0
    for recipeInfo in Recipes:
        moreInfo = recipeInfo.soup.find('div',{'class':'recipe-header__more-info-inner'}).find_all({'i':'class'})
        extraRecipeInfo = [i['class'][1].replace('font-icon-','').replace('-extra-small','') for i in moreInfo]
        
        Recipes[cnt].extraRecipeInfo = extraRecipeInfo
        cnt +=1
        
    return Recipes

In [15]:
def GetRecipeRatings(Recipes):
    cnt=0
    for recipeInfo in Recipes:
        
        Nraters = int(recipeInfo.soup.find(
            'div', {'class': 'rate-info'}).text.strip(' ').replace('(','').replace(' ratings)','') ) 
        if Nraters!=0:
            AggRating = float(recipeInfo.soup.find('meta', {'itemprop': 'ratingValue'})['content'])
            RatersNRatings = [Nraters, AggRating]
            #print(RatersNRatings)
        else:
            RatersNRatings = [Nraters, np.nan]
        
        Recipes[cnt].RatersNRatings = RatersNRatings
        cnt +=1
        
    return Recipes

In [16]:
def GetAuthorInfo(Recipes):
    cnt=0
    for recipeInfo in Recipes:
        
        authorInfoExist = recipeInfo.soup.find('div',{'class':'recipe-header__chef recipe-header__chef--second'})!=None

        if authorInfoExist:
            authorInfo = recipeInfo.soup.find('div',{'class':'recipe-header__chef recipe-header__chef--second'}).find(
            'span',{'class':'author'}).text
            #print(authorInfo)
        else:
            authorInfo = np.nan
            #print('none')
            
        Recipes[cnt].authorInfo = authorInfo
        cnt +=1
        
    return Recipes

In [17]:
## 24-26Nov2016 added

def GetRecipeImgLink(Recipes):
    cnt=0
    for recipeInfo in Recipes:
        imgURL = recipeInfo.soup.find('div',{'class': 'img-container'}).find_all({'img':'src'})[0]['src']
        
        Recipes[cnt].imgURL = imgURL
        cnt +=1
        
    return Recipes

In [18]:
scrapeInt = [int(i) for i in np.round(np.linspace(0, 10081,100))]

In [19]:
sys.setrecursionlimit(1000000) # else pickle would not pickle.... =S

In [20]:
## 24-26Nov2016 || 12 Feb 2017 -- re-ran
folderpath='/Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes/bbc_goodfood_recipes_pickles/'

In [21]:
# for sI in range(33,100): #range(1,len(scrapeInt)): #80,len(scrapeInt)
for sI in range(1,100): #range(1,len(scrapeInt)): #80,len(scrapeInt)
    filename = (folderpath +"scrapedInfo_BBCgdfd_" + format(scrapeInt[sI-1] ) + "to" + format(scrapeInt[sI] ) + "recipes.p")
    #print(filename)
    
    print(format(sI) +': CleanupIngredients&Intructions | GET PrepCookFullTime | RecipeInfo | Ratings | Author etc.') #== pickle-load : ' +filename)
    
    with open(filename, "rb") as f:
        Recipes = pickle.load(f)
    
#     print(format(sI) +': CleanupIngredients&Intructions') #== pickle-load : ' +filename)
    CleanupRecipeIngredients0(Recipes); #updated
    CleanupRecipeIngredients(Recipes); #updated
    CleanupRecipeInstructions(Recipes);

#     print(format(sI) +': GET PrepCookFullTime | RecipeInfo | Ratings | Author etc.') #== pickle-load : ' +filename)
    checkNgetPrepCookFullTimesInfo(Recipes);
    
    GetRecipeDifficulty(Recipes);
    GetRecipeYield(Recipes);
    GetExtraRecipeInfo(Recipes);
    
    GetRecipeRatings(Recipes);
    GetAuthorInfo(Recipes);
    
    GetRecipeImgLink(Recipes);
    
    with open(filename, "wb") as f:
        pickle.dump(Recipes, f)
    print('Cleanedup -- pickle-dumped : ' +filename)

1: CleanupIngredients&Intructions | GET PrepCookFullTime | RecipeInfo | Ratings | Author etc.
Cleanedup -- pickle-dumped : /Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes/bbc_goodfood_recipes_pickles/scrapedInfo_BBCgdfd_0to102recipes.p
2: CleanupIngredients&Intructions | GET PrepCookFullTime | RecipeInfo | Ratings | Author etc.
Cleanedup -- pickle-dumped : /Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes/bbc_goodfood_recipes_pickles/scrapedInfo_BBCgdfd_102to204recipes.p
3: CleanupIngredients&Intructions | GET PrepCookFullTime | RecipeInfo | Ratings | Author etc.
Cleanedup -- pickle-dumped : /Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes/bbc_goodfood_recipes_pickles/scrapedInfo_BBCgdfd_204to305recipes.p
4: CleanupIngredients&Intructions | GET PrepCookFullTime | RecipeInfo | Ratings | Author etc.
Cleanedup -- pickle-dumped : /Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes/bbc_goodfood_recipes_pickles/scrapedInfo_BBCgdfd_305to407recipes.p
5: CleanupIngredients&Intructi

In [ ]:
# filename

In [ ]:
# with open(filename, "rb") as f:
#         Recipes = pickle.load(f)